In [18]:
# imports required to run the code.
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

import pandas as pd
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
import sklearn

In [19]:
# preparing dataset.

dfo = pd.read_csv('healthcare-dataset-stroke-data.csv')
dfo = dfo.fillna(0)

# ---------------------------------------- #

dfo.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,0.0,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [20]:
# preparing dataset.

dfo = pd.read_csv('healthcare-dataset-stroke-data.csv')
dfo = dfo.fillna(0)

dfo = dfo.sort_values(by=['bmi'], ascending=True) # what happened here..?
dfo = dfo.reset_index(drop = True)

# ---------------------------------------- #

df = pd.get_dummies(dfo) # converte dados não numéricos.
df = df.drop(columns=['gender_Other', 'gender_Male'])
df.rename(columns={"gender_Female": "gender"}, inplace = True)
df = df.replace(to_replace=0,value=-1)

# ---------------------------------------- #
df.head()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender,ever_married_No,ever_married_Yes,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,34248,50.0,1,-1,81.96,-1.0,-1,255,1,255,...,255,1,255,255,255,1,255,1,255,255
1,29224,30.0,-1,-1,91.23,-1.0,-1,255,255,1,...,255,1,255,255,255,1,255,255,255,1
2,49894,78.0,1,1,206.53,-1.0,-1,1,255,1,...,255,1,255,255,1,255,255,255,1,255
3,37526,68.0,1,1,233.30,-1.0,-1,1,255,1,...,255,1,255,255,1,255,1,255,255,255
4,29095,71.0,1,-1,93.60,-1.0,-1,255,255,1,...,255,255,1,255,1,255,255,255,1,255


In [21]:
dfa = df.drop(columns=['stroke','id']).astype(float)
# dfa = dfa.drop(columns=['work_type_Never_worked','work_type_Private','work_type_Self-employed','work_type_children'])

X_train, X_test, y_train, y_test = train_test_split(dfa, df['stroke'], train_size=0.5)

y_train.value_counts()

-1    2431
 1     124
Name: stroke, dtype: int64

In [22]:
X_train,X_test,y_train,y_test = X_train.to_numpy(),X_test.to_numpy(),y_train.to_numpy(),y_test.to_numpy()

In [31]:
import autograd.numpy as np_
import numpy as np
from autograd import grad

# X_train, X_test, y_train, y_test

def loss( parametros ):
    w, b, pontos, val = parametros
    est = w.T @ pontos + b
    mse = np_.mean( (est - val)**2)
    return mse

g = grad(loss)

pontos = X_train.T
alvos = (y_train).astype(float)

w = np.random.randn( pontos.shape[0],1)
b = 0.0
alpha = 0.000001

for n in range(100000):
    grad_ = g( (w, b, pontos, alvos) )
    w -= alpha*grad_[0]
    b -= alpha*grad_[1]

# ---------------------------------------- #

w,b

(array([[-3.41117485e-03],
        [ 9.89539948e-01],
        [ 4.46520433e-01],
        [-9.54129792e-04],
        [-1.32972531e-03],
        [-3.62872752e-04],
        [-4.39712837e-03],
        [-4.39441578e-03],
        [-7.45502339e-01],
        [-7.45538723e-01],
        [-7.45588055e-01],
        [-7.45496403e-01],
        [-7.45265571e-01],
        [-7.60902529e-01],
        [-7.61043227e-01],
        [ 1.24993779e+00],
        [ 1.25024893e+00],
        [ 1.25040932e+00],
        [ 1.25035473e+00]]),
 -0.0019268904353147629)

In [32]:
def predict( w, b, pontos ):
    est = w.T @ pontos + b
    return est

y_pred = predict( w, b, X_test.T )

# ---------------------------------------- #

def accuracy(y_test, y_est):
    return np.mean(np.sign(y_test)==np.sign(y_est))

print(y_test.shape,y_pred.shape)

accuracy(y_test, y_pred)

(2555,) (1, 2555)


0.8712328767123287

In [25]:
import pandas as pd
import matplotlib.pyplot as plt

# dfa.plot(subplots=True, figsize=(22,10))
# plt.show()

In [26]:
# Função para avaliar o classificador.
# sklearn.metrics.accuracy_score(y_true, y_pred)

In [27]:
# ---------------------------------------- #

# df_rotulo = df['id']
# df_features = df[['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status', 'stroke']]

# tree = DecisionTreeClassifier(criterion='entropy')
# tree.fit(df_features, df_rotulo)

# ---------------------------------------- #

In [28]:
# from sklearn.tree import plot_tree
# import matplotlib.pyplot as plt

# plt.figure( figsize=(20,20) )
# a = plot_tree(tree, feature_names=df_features.columns, fontsize=2, 
#               node_ids=False, impurity=False, filled=True)